# Chapter 1: Basics and Linear Models

In [1]:
# Chapter 1 exercise imports and data
import numpy as np
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.linear_model import LinearRegression, GammaRegressor
from sklearn.preprocessing import OneHotEncoder, FunctionTransformer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline
from sklearn.metrics import root_mean_squared_error

diamonds = pd.read_parquet("diamonds.parquet")  # Or sns.load_dataset("diamonds")

ord_vars = ["color", "cut", "clarity"]
ord_levels = [diamonds[x].cat.categories.to_list() for x in ord_vars]

diamonds.head()

,carat,cut,color,clarity,depth,table,price,x,y,z
0,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
1,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
2,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31
3,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63
4,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75


## Exercise on linear regression

In [2]:
# Via scikit-learn
y = diamonds["price"]

model = make_pipeline(
    ColumnTransformer(
        transformers=[
            ("linear", "passthrough", ["carat"]),
            ("dummies", OneHotEncoder(categories=ord_levels, drop="first"), ord_vars),
        ],
        verbose_feature_names_out=False,
    ),
    LinearRegression(),
)
model.fit(diamonds, y)

rmse = root_mean_squared_error(y, model.predict(diamonds))
print(f"RMSE: {rmse:.3f}")
print(f"R-squared: {model.score(diamonds, y):.2%}")
print("Intercept", model[-1].intercept_)

results = pd.DataFrame(
    model[-1].coef_, columns=["Estimates"], index=model[:-1].get_feature_names_out()
)
results

RMSE: 1156.648
R-squared: 91.59%
Intercept -944.9008751128845


,Estimates
carat,8886.128884
color_E,-211.682482
color_F,-303.310033
color_G,-506.199536
color_H,-978.697664
color_I,-1440.301902
color_J,-2325.222360
cut_Premium,-128.858534
cut_Very Good,-149.537561
cut_Good,-342.486990


In [3]:
# Via statsmodels
model2 = smf.ols("price ~ carat + color + cut + clarity", data=diamonds).fit()
model2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.916
Model:                            OLS   Adj. R-squared:                  0.916
Method:                 Least Squares   F-statistic:                 3.264e+04
Date:                Sat, 17 Feb 2024   Prob (F-statistic):               0.00
Time:                        15:32:46   Log-Likelihood:            -4.5699e+05
No. Observations:               53940   AIC:                         9.140e+05
Df Residuals:                   53921   BIC:                         9.142e+05
Df Model:                          18                                         
Covariance Type:            nonrobust                                         
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
Intercept         -944.9009     31.395    -30.098      0.000   -1006.434    -883.367
color[T.E]        -211.6825     18.316    -11.557      0.000    -247.582    -175.783
color[T.F]        -303.3100     18.509    -16.387      0.000    -339.589    -267.031
color[T.G]        -506.1995     18.122    -27.933      0.000    -541.719    -470.680
color[T.H]        -978.6977     19.272    -50.784      0.000   -1016.471    -940.925
color[T.I]       -1440.3019     21.646    -66.538      0.000   -1482.729   -1397.875
color[T.J]       -2325.2224     26.723    -87.013      0.000   -2377.599   -2272.846
cut[T.Premium]    -128.8585     12.894     -9.994      0.000    -154.131    -103.586
cut[T.Very Good]  -149.5376     13.265    -11.273      0.000    -175.538    -123.537
cut[T.Good]       -342.4870     18.524    -18.489      0.000    -378.794    -306.180
cut[T.Fair]       -998.2544     30.656    -32.563      0.000   -1058.341    -938.168
clarity[T.VVS1]   -347.6192     33.440    -10.395      0.000    -413.162    -282.076
clarity[T.VVS2]   -452.4474     31.951    -14.160      0.000    -515.072    -389.822
clarity[T.VS1]    -884.7679     30.456    -29.050      0.000    -944.463    -825.073
clarity[T.VS2]   -1201.8177     29.721    -40.436      0.000   -1260.072   -1143.564
clarity[T.SI1]   -1845.9589     29.857    -61.827      0.000   -1904.479   -1787.439
clarity[T.SI2]   -2793.6969     31.085    -89.873      0.000   -2854.624   -2732.770
clarity[T.I1]    -5419.6468     52.136   -103.952      0.000   -5521.834   -5317.460
carat             8886.1289     12.034    738.437      0.000    8862.543    8909.715
==============================================================================
Omnibus:                    15285.474   Durbin-Watson:                   0.907
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           183262.957
Skew:                           1.022   Prob(JB):                         0.00
Kurtosis:                      11.796   Cond. No.                         24.2
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [4]:
print(f"RMSE: {root_mean_squared_error(y, model2.predict(diamonds)):.3f}")

RMSE: 1156.648


**Comments**

- **Model quality:** About 92% of price variations are explained by covariates. Typical prediction error is 1157 USD.
- **Effects:** All effects point into the intuitively right direction (larger stones are more expensive, worse color are less expensive etc.)
- **Practical perspective:** Additivity in color, cut and clarity are not making sense. Their effects should get larger with larger diamond size. This can be solved by adding interaction terms with carat or, much easier, to switch to a logarithmic response.

## Exercise on GLMs

In [5]:
# Via statsmodels
model = smf.glm(
    "price ~ np.log(carat) + color + cut + clarity",
    data=diamonds,
    family=sm.families.Gamma(sm.families.links.Log()),
).fit()

print(model.summary())

bias = diamonds["price"].mean() / model.predict(diamonds).mean() - 1
print(f"Relative bias on USD scale: {bias:.3%}")

                 Generalized Linear Model Regression Results                  
Dep. Variable:                  price   No. Observations:                53940
Model:                            GLM   Df Residuals:                    53921
Model Family:                   Gamma   Df Model:                           18
Link Function:                    Log   Scale:                        0.019471
Method:                          IRLS   Log-Likelihood:            -3.8857e+05
Date:                Sat, 17 Feb 2024   Deviance:                       978.68
Time:                        15:32:48   Pearson chi2:                 1.05e+03
No. Iterations:                    12   Pseudo R-squ. (CS):              1.000
Covariance Type:            nonrobust                                         
                       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------
Intercept            9.1446      0.004  

In [6]:
# Via scikit-learn
y = diamonds["price"]

# Define log_carat outside pipeline as it is tricky to track feature names
diamonds["log_carat"] = np.log(diamonds["carat"])

# Define and fit model pipeline. Note: GammaRegressor directly uses log-link
take_log = FunctionTransformer(np.log, feature_names_out="one-to-one")
model2 = make_pipeline(
    ColumnTransformer(
        transformers=[
            ("log_carat", take_log, ["carat"]),
            ("dummies", OneHotEncoder(categories=ord_levels, drop="first"), ord_vars),
        ],
        verbose_feature_names_out=False,
    ),
    GammaRegressor(alpha=0, solver="newton-cholesky"),
)
model2.fit(diamonds, y)

# Performance
d2 = model2.score(diamonds, y)
print(f"Percent deviance explained: {d2:.2%}")

# Relative bias
bias2 = y.mean() / model2.predict(diamonds).mean() - 1
print(f"Relative bias on USD scale: {bias2:.3%}")

# Fitted coefficients
print("Intercept", model2[-1].intercept_)
results = pd.DataFrame(
    model2[-1].coef_, columns=["Estimates"], index=model2[:-1].get_feature_names_out()
)
results

d:\ml_lecture\.venv\Lib\site-packages\joblib\externals\loky\backend\context.py:136: UserWarning: Could not find the number of physical cores for the following reason:
found 0 physical cores < 1
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.
  warnings.warn(
  File "d:\ml_lecture\.venv\Lib\site-packages\joblib\externals\loky\backend\context.py", line 282, in _count_physical_cores
    raise ValueError(f"found {cpu_count_physical} physical cores < 1")


Percent deviance explained: 98.15%
Relative bias on USD scale: 0.336%
Intercept 9.144584407895891


,Estimates
carat,1.881556
color_E,-0.056294
color_F,-0.096366
color_G,-0.163527
color_H,-0.253498
color_I,-0.374275
color_J,-0.511088
cut_Premium,-0.020745
cut_Very Good,-0.044901
cut_Good,-0.081495


**Comment:** The coefficients are very similar to the linear regression with log(price) as response. This makes sense as we interpret the coefficients in the same way! The bias is only 0.3%, i.e., much smaller than the 3% of the OLS with log(price) as response. Still, because the log is not the natural link of the Gamma regression, the bias is not exactly 0.